In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['HF_HOME'] = '/data/models/huggingface'

### 1. 加载模型和分词器

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

### 2. 加载数据集

In [7]:
from datasets import load_dataset
datasets = load_dataset("Mxode/Chinese-Instruct", "dpsk-r1-distil")
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'response'],
        num_rows: 100331
    })
})

In [8]:
# 原始数据集只包含 train， 我们需要将其分割为 train 和 test
# 使用 train_test_split 方法进行分割
split_datasets = datasets['train'].train_test_split(test_size=0.3, seed=42)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'prompt', 'response'],
        num_rows: 70231
    })
    test: Dataset({
        features: ['id', 'prompt', 'response'],
        num_rows: 30100
    })
})

In [9]:
split_datasets['train'][0], split_datasets['test'][0]

({'id': 'Gn4-4c5oTpqe',
  'prompt': '请问“好矮你鸭”这个梗是怎么来的？',
  'response': '“好矮你鸭”是近期在短视频平台（如抖音）兴起的谐音梗，源自一位网友模仿广东口音说“好爱你呀”的搞笑视频。具体解析如下：\n\n1. 语言机制：\n- "好矮"对应普通话"好爱"的广式发音（受粤语腔调影响，尾音下沉）\n- "你鸭"是"你呀"的萌化变体，常见于网络用语\n\n2. 传播路径：\n2022年3月某用户发布变声对口型视频，用夸张的广东腔调说情话，其中"好爱你呀"因发音魔性引发二创热潮，播放量超8000万次。网友在评论区故意用谐音文字复刻笑点，逐步形成固定梗。\n\n3. 使用场景：\n- 情侣间反套路表白（例如女生吐槽男友身高后突然说"好矮你鸭"）\n- 玩身高梗时作为双关语（既调侃身高又表达亲昵）\n- 方言模仿类视频的常见弹幕互动\n\n4. 文化现象：\n该梗反映了Z世代对传统情话的解构，通过"土味发音+文字游戏"的方式消解严肃表达，符合年轻人"用戏谑传递真情"的社交习惯。同时展现了方言在网络文化中的再创作活力。\n\n注：该梗近期热度已有所下降，逐渐被"这是可以说的吗"等新梗替代，建议使用时注意语境以免造成交流障碍。'},
 {'id': 'f5VDS7z-XYgp',
  'prompt': '请解释NaCl/Triton X-100相互作用的物理化学机理',
  'response': 'NaCl与Triton X-100（一种非离子型表面活性剂）的相互作用涉及多种物理化学机制，主要包括盐对表面活性剂临界胶束浓度（CMC）、胶束结构和溶液热力学性质的影响。以下是其机理的分步解释：\n\n---\n\n### 1. **盐析效应（Salting-out Effect）**\n- **原理**：NaCl的加入会降低Triton X-100的溶解度，促使其在更低浓度下形成胶束（即降低CMC）。\n- **机制**：\n  - **离子水合作用**：Na⁺和Cl⁻离子在水中强烈水合，争夺自由水分子，减少了可用于溶解Triton X-100的水分子数量。\n  - **疏水作用增强**：盐离子通过改变水的氢键网络（“水结构”），增强疏水效应，使Triton X-100的疏水尾部更易聚集形成胶束。\n\n---\n

#### 将数据集数据 token 化

In [10]:
# 查看 tokenizer 的 chat_template
print(tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0].role == 'system' %}
        {{- messages[0].content + '\n\n' }}
    {%- endif %}
    {{- "# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0].role == 'system' %}
        {{- '<|im_start|>system\n' + messages[0].content + '<|im_end|>\n' }}
    {%- endif %}
{%- endif %}
{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}
{%- for message in messages[::-1] %}
    {%- set index = (messages|length - 

In [ ]:
# 将数据集转换为 Qwen3 template格式
# Qwen3 的 template 格式是的一个最小的对话格式，例如：
# [
#     {"role": "user", "content": "你好"},
#     {"role": "assistant", "content": "你好，有什么可以帮助你的吗？"}
# ]
def convert_to_qwen3_format(system: str = "You are a helpful assistant.", user: str = "", assistant: str = ""):
    return [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
        {"role": "assistant", "content": assistant}
    ]



# # 定义一个函数来处理数据集中的每个样本
# def tokenize_function(example):
#     # 检查 prompt 和 response 是否为字符串或列表(datasets.map 中 batch=True 时会将 prompt 和 response 转换为列表)
#     if isinstance(example['prompt'], str) and isinstance(example['response'], str):
#         qwen3_format_list = convert_to_qwen3_format(example['prompt'], example['response'])

#     if isinstance(example['prompt'], list) and isinstance(example['response'], list):
#         qwen3_format_list = [
#             convert_to_qwen3_format(prompt, response)
#             for prompt, response in zip(example['prompt'], example['response'])
#         ]

#     text = tokenizer.apply_chat_template(qwen3_format_list, tokenize=False, add_generation_prompt=False)
#     # # 使用 tokenizer 编码, 不进行填充,后面会在构建训练 batch 时进行填充
#     # # 这里的 max_length 可以根据需要调整
#     # # 注意：如果文本长度超过 max_length，tokenizer 会截断文本
#     return tokenizer(text, truncation=True, max_length=1024)

def tokenize_function(example):
    # 构造 Qwen3 对话格式
    
    
    qwen3_format_list = convert_to_qwen3_format(example.get('system', None), example.get('user', None), example.get('assistant', None))

    # 生成完整的 chat 文本（不添加 generation prompt）
    text = tokenizer.apply_chat_template(
        qwen3_format_list, tokenize=False, add_generation_prompt=False
    )

    input_ids = tokenizer(text, return_tensors="pt").input_ids[0]  # 获取第一个样本的 input_ids

    labels = input_ids.clone()  # 复制 input_ids 作为 labels


    # return {
    #     "input_ids": input_ids,
    #     "attention_mask": tokenized["attention_mask"],
    #     "labels": labels
    # }




In [ ]:
# 使用 map 函数对数据集进行处理
# 会将 tokenize_function 返回的结果添加到数据集的字段中
split_datasets = split_datasets.map(
    tokenize_function,
    # batched=True,  # 批处理模式
    remove_columns=split_datasets['train'].column_names,  # 移除原有的列，只保留 tokenized 的结果
    num_proc=16,  # 并行处理的进程数
    desc="Tokenizing dataset"
)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 70231
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 30100
    })
})

In [9]:
ids = split_datasets["train"][0]["input_ids"]  # tokenize 后的输入 ID
tokenizer.decode(ids, skip_special_tokens=False)  # 解码查看文本内容

'<|im_start|>user\n请问“好矮你鸭”这个梗是怎么来的？<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n“好矮你鸭”是近期在短视频平台（如抖音）兴起的谐音梗，源自一位网友模仿广东口音说“好爱你呀”的搞笑视频。具体解析如下：\n\n1. 语言机制：\n- "好矮"对应普通话"好爱"的广式发音（受粤语腔调影响，尾音下沉）\n- "你鸭"是"你呀"的萌化变体，常见于网络用语\n\n2. 传播路径：\n2022年3月某用户发布变声对口型视频，用夸张的广东腔调说情话，其中"好爱你呀"因发音魔性引发二创热潮，播放量超8000万次。网友在评论区故意用谐音文字复刻笑点，逐步形成固定梗。\n\n3. 使用场景：\n- 情侣间反套路表白（例如女生吐槽男友身高后突然说"好矮你鸭"）\n- 玩身高梗时作为双关语（既调侃身高又表达亲昵）\n- 方言模仿类视频的常见弹幕互动\n\n4. 文化现象：\n该梗反映了Z世代对传统情话的解构，通过"土味发音+文字游戏"的方式消解严肃表达，符合年轻人"用戏谑传递真情"的社交习惯。同时展现了方言在网络文化中的再创作活力。\n\n注：该梗近期热度已有所下降，逐渐被"这是可以说的吗"等新梗替代，建议使用时注意语境以免造成交流障碍。<|im_end|>\n'

#### 数据集 batch 化处理
涉及 batch 数据 padding， 生成 batch 数据的 tensor

In [ ]:
# 使用 DataCollatorForLanguageModeling 来处理数据集
from transformers import DataCollatorForLanguageModeling, DataCollatorForSeq2Seq
# 需要传入 tokenizer 来让其知道填充的 token、 padding的方式
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    pad_to_multiple_of=8,
)

In [11]:
# 查看数据集的一个 batch
batch_samples = [split_datasets["train"][i] for i in range(4)]
[len(sample['input_ids']) for sample in batch_samples]  # 查看 batch 每个样本的长度

NameError: name 'split_datasets' is not defined

In [ ]:
# 使用 data_collator 对 batch 进行处理
# 这将会对 batch 中的样本进行填充，并返回一个字典
# 包含 'input_ids' 和 'attention_mask' 等字段
batch_collated = data_collator(batch_samples)
(batch_collated['input_ids'].shape, batch_collated)  # 查看填充后的 input_ids 的形状和内容

(torch.Size([4, 598]),
 {'input_ids': tensor([[151644,    872,    198,  ..., 151643, 151643, 151643],
         [151644,    872,    198,  ..., 151643, 151643, 151643],
         [151644,    872,    198,  ..., 151643, 151643, 151643],
         [151644,    872,    198,  ...,   1773, 151645,    198]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[151644,    872,    198,  ...,   -100,   -100,   -100],
         [151644,    872,    198,  ...,   -100,   -100,   -100],
         [151644,    872,    198,  ...,   -100,   -100,   -100],
         [151644,    872,    198,  ...,   1773, 151645,    198]])})

In [13]:
tokenizer.decode(batch_collated['input_ids'][0], skip_special_tokens=False)  # 解码查看第一个样本的文本内容 padding token 会被解码为 <|endoftext|>，可以看到填充的部分

'<|im_start|>user\n请问“好矮你鸭”这个梗是怎么来的？<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n“好矮你鸭”是近期在短视频平台（如抖音）兴起的谐音梗，源自一位网友模仿广东口音说“好爱你呀”的搞笑视频。具体解析如下：\n\n1. 语言机制：\n- "好矮"对应普通话"好爱"的广式发音（受粤语腔调影响，尾音下沉）\n- "你鸭"是"你呀"的萌化变体，常见于网络用语\n\n2. 传播路径：\n2022年3月某用户发布变声对口型视频，用夸张的广东腔调说情话，其中"好爱你呀"因发音魔性引发二创热潮，播放量超8000万次。网友在评论区故意用谐音文字复刻笑点，逐步形成固定梗。\n\n3. 使用场景：\n- 情侣间反套路表白（例如女生吐槽男友身高后突然说"好矮你鸭"）\n- 玩身高梗时作为双关语（既调侃身高又表达亲昵）\n- 方言模仿类视频的常见弹幕互动\n\n4. 文化现象：\n该梗反映了Z世代对传统情话的解构，通过"土味发音+文字游戏"的方式消解严肃表达，符合年轻人"用戏谑传递真情"的社交习惯。同时展现了方言在网络文化中的再创作活力。\n\n注：该梗近期热度已有所下降，逐渐被"这是可以说的吗"等新梗替代，建议使用时注意语境以免造成交流障碍。<|im_end|>\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|en

### 3. 开始训练

In [14]:
# 设置训练参数
from transformers import TrainingArguments
output_dir = "./qwen3-finetuned"
training_args = TrainingArguments(
    output_dir=output_dir,  # 模型保存的目录
    num_train_epochs=1,  # 训练的轮数
    learning_rate=2e-5,  # 学习率
    warmup_steps=100,  # 预热步数
    weight_decay=0.1,  # 权重衰减
    per_device_train_batch_size=16,  # 训练时的 batch size
    per_device_eval_batch_size=16,  # 评估时的 batch size
    # gradient_checkpointing=True,  # 启用梯度检查点以节省内存
    gradient_accumulation_steps=8,  # 梯度累积步数
    logging_dir=os.path.join(output_dir, "logs"),  # 日志目录
    logging_steps=5,  # 每5步记录一次日志
    bf16=True,  # 或 fp16=True, 看你的GPU支持
    report_to="swanlab",  # 使用 SwanLab 进行日志记录
    dataloader_num_workers=16,  # 数据加载的工作线程数
    run_name="qwen3-finetuned-chinese-instruct",  # 训练任务的名称(在 SwanLab 中显示)
    save_strategy="epoch", 
    save_total_limit=3,  # 最多保存3个模型
    save_steps=200,  # 每200步保存一次模型
    eval_steps=10,  # 每10步进行一次评估
)

In [15]:
from sklearn.metrics import accuracy_score
import json
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # 将预测结果和标签转换为 numpy 数组
    predictions = predictions.argmax(axis=-1)
    labels = labels.flatten()
    # 计算准确率
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

In [ ]:
# 使用 Trainer 进行训练
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_datasets["train"],
    eval_dataset=split_datasets["test"],  # 使用分割后的测试集进行评估
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics  # 添加自定义的评估函数
)

[2025-06-25 16:42:07,185] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/miniconda3/envs/easydistill/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/miniconda3/envs/easydistill/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/miniconda3/envs/easydistill/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/miniconda3/envs/easydistill/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/miniconda3/envs/easydistill/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/opt/miniconda3/envs/easydistill/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/opt/miniconda3/envs/easydistill

[2025-06-25 16:42:08,451] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


In [ ]:
train_results = trainer.train()  # 开始训练
print(train_results)
trainer.log_metrics("train", train_results.metrics)

# 保存模型和 tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

swanlab: \ Waiting for the swanlab cloud response.

swanlab: Tracking run with swanlab version 0.6.4                                   
swanlab: Run data will be saved locally in /root/zhiyuan/workspace/AiO/train/transformers/train/swanlog/run-20250625_164210-6c031199
swanlab: 👋 Hi what_fuck, welcome to swanlab!
swanlab: Syncing run qwen3-finetuned-chinese-instruct to the cloud
swanlab: 🏠 View project at https://swanlab.cn/@what_fuck/train
swanlab: 🚀 View run at https://swanlab.cn/@what_fuck/train/runs/rs973ythl3lcpmbj8ygyd


Step,Training Loss
5,2.170900
10,2.129100
15,2.134600
20,2.083200
25,2.083000
30,2.044600
35,1.921500
40,1.869900
45,1.895600
50,1.862600


TrainOutput(global_step=549, training_loss=1.7766801368560514, metrics={'train_runtime': 3820.1092, 'train_samples_per_second': 18.385, 'train_steps_per_second': 0.144, 'total_flos': 1.8667264935002112e+17, 'train_loss': 1.7766801368560514, 'epoch': 1.0})
***** train metrics *****
  epoch                    =         1.0
  total_flos               = 173852452GF
  train_loss               =      1.7767
  train_runtime            =  1:03:40.10
  train_samples_per_second =      18.385
  train_steps_per_second   =       0.144


('./qwen3-finetuned/tokenizer_config.json',
 './qwen3-finetuned/special_tokens_map.json',
 './qwen3-finetuned/chat_template.jinja',
 './qwen3-finetuned/vocab.json',
 './qwen3-finetuned/merges.txt',
 './qwen3-finetuned/added_tokens.json',
 './qwen3-finetuned/tokenizer.json')

: 